<a href="https://colab.research.google.com/github/AfshinRezakhani/Thesis1/blob/main/GAN_GRU_d5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Reshape, Input
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

file_path = "/content/d5.csv"
df = pd.read_csv(file_path)

target_column = "F"  # ستون هدف
features = df.drop(columns=[target_column]).values
target = df[target_column].values

print("🔹 توزیع کلاس‌های اصلی در داده‌ها:", np.unique(target, return_counts=True))

scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

def build_generator():
    model = Sequential([
        Input(shape=(1, X_train.shape[2])),
        GRU(128, return_sequences=True),
        Dropout(0.2),
        GRU(64, return_sequences=True),
        Dropout(0.2),
        GRU(32, return_sequences=False),
        Dense(X_train.shape[2]),
        Reshape((1, X_train.shape[2]))
    ])
    return model

def build_discriminator():
    model = Sequential([
        Input(shape=(1, X_train.shape[2])),
        GRU(128, return_sequences=True),
        Dropout(0.3),
        GRU(64, return_sequences=False),
        Dense(1, activation='sigmoid')
    ])
    return model

def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential([generator, discriminator])
    return model

discriminator = build_discriminator()
discriminator.compile(loss="binary_crossentropy", optimizer=Adam(0.0002, 0.5), metrics=["accuracy"])

generator = build_generator()
gan = build_gan(generator, discriminator)
gan.compile(loss="binary_crossentropy", optimizer=Adam(0.0002, 0.5))

epochs = 1000
batch_size = 128
half_batch = batch_size // 2

for epoch in range(epochs):
    noise = np.random.normal(0, 1, (half_batch, 1, X_train.shape[2]))
    generated_data = generator.predict(noise, verbose=0)

    idx = np.random.randint(0, X_train.shape[0], half_batch)
    real_data = X_train[idx]

    real_labels = np.random.uniform(0.8, 1.0, (half_batch, 1))
    fake_labels = np.random.uniform(0.0, 0.2, (half_batch, 1))

    X_combined = np.concatenate((real_data, generated_data))
    y_combined = np.concatenate((real_labels, fake_labels))

    if epoch % 2 == 0:
        d_loss = discriminator.train_on_batch(X_combined, y_combined)

    noise = np.random.normal(0, 1, (batch_size, 1, X_train.shape[2]))
    y_mislabeled = np.ones((batch_size, 1))

    g_loss = gan.train_on_batch(noise, y_mislabeled)

    d_loss = float(d_loss[0]) if isinstance(d_loss, list) else float(d_loss)
    g_loss = float(g_loss[0]) if isinstance(g_loss, list) else float(g_loss)

    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Discriminator Loss: {d_loss:.4f}, Generator Loss: {g_loss:.4f}")

minority_class = 0 if np.bincount(y_train)[0] < np.bincount(y_train)[1] else 1
num_samples_to_generate = abs(np.bincount(y_train)[0] - np.bincount(y_train)[1])

noise = np.random.normal(0, 1, (num_samples_to_generate, 1, X_train.shape[2]))
generated_samples = generator.predict(noise, verbose=0)

synthetic_labels = np.full((num_samples_to_generate,), minority_class)

X_balanced = np.concatenate((X_train.squeeze(), generated_samples.squeeze()))
y_balanced = np.concatenate((y_train, synthetic_labels))

df_balanced = pd.DataFrame(X_balanced)
df_balanced.insert(0, target_column, y_balanced)  # قرار دادن ستون هدف در ستون اول

balanced_file_path = "/content/d5_balancedwithGRU.csv"
df_balanced.to_csv(balanced_file_path, index=False)
print(f"✅ Balanced dataset saved as {balanced_file_path}")

y_pred_probs = discriminator.predict(X_test, batch_size=512, verbose=1)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

print("🔹 توزیع کلاس‌ها در پیش‌بینی مدل:", np.unique(y_pred, return_counts=True))

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"✅ Accuracy: {accuracy:.4f}")
print(f"✅ F1-Score: {f1:.4f}")


🔹 توزیع کلاس‌های اصلی در داده‌ها: (array([0, 1]), array([ 895, 9105]))


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0: Discriminator Loss: 0.6832, Generator Loss: 0.6931
Epoch 100: Discriminator Loss: 0.6865, Generator Loss: 0.6834
Epoch 200: Discriminator Loss: 0.7117, Generator Loss: 0.6318
Epoch 300: Discriminator Loss: 0.7352, Generator Loss: 0.5888
Epoch 400: Discriminator Loss: 0.7501, Generator Loss: 0.5622
Epoch 500: Discriminator Loss: 0.7607, Generator Loss: 0.5438
Epoch 600: Discriminator Loss: 0.7690, Generator Loss: 0.5301
Epoch 700: Discriminator Loss: 0.7757, Generator Loss: 0.5192
Epoch 800: Discriminator Loss: 0.7813, Generator Loss: 0.5103
Epoch 900: Discriminator Loss: 0.7860, Generator Loss: 0.5027
✅ Balanced dataset saved as /content/d5_balancedwithGRU.csv
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step
🔹 توزیع کلاس‌ها در پیش‌بینی مدل: (array([1]), array([2000]))
✅ Accuracy: 0.9015
✅ F1-Score: 0.9482
